# P8 Fruits - complément 

## Installations 

In [ ]:
# ! pip install --upgrade pip
! pip  install pyspark
! pip install intel-tensorflow

#! pip install tensorflow

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.appName("Image_app").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/21 17:47:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import boto3
import io
import requests
import time
import pandas as pd
from PIL import Image
import numpy as np 
import io

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

from pyspark.sql.functions import col, pandas_udf, PandasUDFType




## Chargement des informations à partir de S3 

In [15]:
# create an S3 client
s3 = boto3.client("s3")

### Création du dataframe image

In [16]:
from pyspark.sql.types import *

# Define the schema for the DataFrame
schema = StructType([
    StructField("image", BinaryType()),
    StructField("label", StringType())
])

# Create the empty DataFrame with the specified schema
images_df = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)
images_df.printSchema() 

root
 |-- image: binary (nullable = true)
 |-- label: string (nullable = true)



In [17]:
columns = ['image', 'label']

for fruit in ['pear','carrot','zucchini','apple','cabbage']:
    #récuperer la liste des fichier dans le Bucket S3 
    response = s3.list_objects_v2(Bucket='bucket-p8-hh', Prefix=fruit)

    
    for content in response.get('Contents', []):
       
    
        file_path= content['Key']
        url = s3.generate_presigned_url(ClientMethod="get_object", Params={"Bucket": "bucket-p8-hh", "Key": file_path})
        response = requests.get(url)
        newRow = spark.createDataFrame([(response.content,fruit)], columns)
        appended = images_df.union(newRow)
        images_df=appended

In [18]:
images_df = images_df.filter(images_df.label!= 'drop')
#images_df.show()
images_df.printSchema()
images=images_df



root
 |-- image: binary (nullable = true)
 |-- label: string (nullable = true)



## Récupération des features de ResNet50

In [19]:
model = ResNet50(include_top=False)
model.summary()  # verify that the top layer is removed

from pyspark import SparkContext
sc = SparkSession.builder.getOrCreate().sparkContext
bc_model_weights = sc.broadcast(model.get_weights())

def model_fn():
  """
  Returns a ResNet50 model with top layer removed and broadcasted pretrained weights.
  """
  model = ResNet50(weights=None, include_top=False)
  model.set_weights(bc_model_weights.value)
  return model


2023-01-21 17:48:13.995970: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-21 17:48:13.996289: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 


94765736/94765736 [==============================] - 3s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, None, No  0           []                               
                                ne)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 3, None, Non  0           ['input_1[0][0]']                
                                e)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, None, No  9472        ['conv1_pad[0][0]']              
                          

In [20]:
def preprocess(content):
  """
  Preprocesses raw image bytes for prediction.
  """
  img = Image.open(io.BytesIO(content)).resize([224, 224])
  arr = img_to_array(img)
  return preprocess_input(arr)

def featurize_series(model, content_series):
  """
  Featurize a pd.Series of raw images using the input model.
  :return: a pd.Series of image features
  """
  input = np.stack(content_series.map(preprocess))
  preds = model.predict(input)
  # For some layers, output features will be multi-dimensional tensors.
  # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
  output = [p.flatten() for p in preds]
  return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
  '''
  This method is a Scalar Iterator pandas UDF wrapping our featurization function.
  The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
  
  :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
  '''
  # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
  # for multiple data batches.  This amortizes the overhead of loading big models.
  model = model_fn()
  for content_series in content_series_iter:
    yield featurize_series(model, content_series)


/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/pyspark/sql/pandas/functions.py:394: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [71]:
features_df = images.repartition(16).select(col("image"), featurize_udf("image").alias("features"))

In [72]:
features_df.printSchema()

root
 |-- image: binary (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)



## sauvegarde 

In [73]:
from pyspark.sql.functions import posexplode

# Flatten the arrays in the DataFrame

flat_df = features_df.select(posexplode("features").alias("feature_array", "feature_element"))
flat_df.printSchema()

root
 |-- feature_array: integer (nullable = false)
 |-- feature_element: float (nullable = true)



In [77]:
flat_df3 = features_df.selectExpr("image", "posexplode(features) as (feature_index, feature_element)")
flat_df3  = flat_df2 .drop("features")

In [79]:
flat_df3.printSchema()
# flat_df = flat_df3 OOM with flatdf2 qui a  la colonne image en plus 

root
 |-- image: binary (nullable = true)
 |-- feature_index: integer (nullable = false)
 |-- feature_element: float (nullable = true)



In [ ]:
flat_df.count(), len(flat_df.columns)

In [ ]:
flat_df.coalesce(1).write.format("csv").mode("overwrite").save("/home/ec2-user/SageMaker/result")

In [ ]:
import os
# Specify the local directory path
local_dir = '/home/ec2-user/SageMaker/result'

# Get all the files in the directory
files = os.listdir(local_dir)

# Loop through the files
for file in files:

    if file.endswith(".csv"):
        # Rename the file
        new_file = 'feature50_image_result.csv'
        new_file_path = os.path.join(local_dir, file) # # Build the local file path
        s3.upload_file(new_file_path, 'bucket-p8-hh', new_file)      

        
        
        